#**Задача 2.1**
[Выборка:](https://github.com/andriygav/PSAD/blob/master/labs/lab2/data/2.1.csv)

Рассматривается задача тестирования вакцины от некоторого вируса. Производство вакцины очень дорогое и затратное по времени, поэтому в день может быть произведена только одна ампула.

Требуется проверить, что вакцина помогает (вероятность заразиться меньше у человека с вакциной, чем у человека без вакцины).

Эксперимент ставится следующим образом: каждый день есть два идентичных по здоровью человека. Один из людей принимает вакцину, а второй нет, после чего обоих ставят в одну среду с вирусом. В конце для проверяют кто заразился. (В таблице: s --- sick; h --- healthy)

Весь мир ждет вакцину от данного вируса, поэтому к руководству института постоянно приходят запросы о сроках завершения тестирования образца. Руководство поручило Вам оценить среднее время, которое понадобится на тестирования данной вакцины. А также провести анализ полученных данных на уровне значимости $\alpha=0.05$ и при ошибке второго рода $\beta=0.2$.

Требуется:

1. Записать задачу формально;
2. Выполнить оценку среднего количества дней для принятия решения (учесть что истинная вероятность заразиться с вакциной и без равны $p_1 = 0.2, p_2 = 0.5$ соответственно);
3. Выполнить анализ данных и выяснить работает ли вакцина или нет.

Все выкладки должны быть сделаны аналитически, без использования компьютера.



---
#Решение
Задача на последовательный анализ Вальда, т.к. нам хочется дать ответ за меньшее чисо итераций. Сначала посмотрим на данные:



In [3]:
!pip install sprt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sprt: filename=sprt-0.0.1-py3-none-any.whl size=4017 sha256=4e465ed9a2b2aefe5d3f9e8446d95af39af39191903ed6641abd406606eb0477
  Stored in directory: /root/.cache/pip/wheels/57/4f/5e/0ccefece378c1aac79f5e3093849173236b19f3781e070ab03
Successfully built sprt


In [13]:
import numpy as np
import sprt
import scipy
import scipy.stats as st
from statsmodels.stats.proportion import proportions_ztest
import pandas as pd

import matplotlib.pyplot as plt

In [5]:
# параметр для задания толщины линии
plt.rcParams['lines.linewidth'] = 1
# параметр для задания ширины точек
plt.rcParams['lines.markersize'] = 6
# параметр для задания шрифта осей x и y (цифр на осях)
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
# параметр для задания шрифта на осях 
plt.rcParams['legend.fontsize'] = 12
# параметр для задания шрифта заголовка
plt.rcParams['axes.titlesize'] = 32
# параметр для задания шрифта осей x и y (название оси)
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['figure.figsize'] = (8.0, 6.0)
# параметр для задания шрифта вспомагательных записей на графику (annotate или text)
plt.rcParams['font.size'] = 13

In [27]:
data_pre = pd.read_csv("https://raw.githubusercontent.com/andriygav/PSAD/master/labs/lab2/data/2.1.csv", delimiter='\,')
data_pre.head(20)

<ipython-input-27-130949dce808>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_pre = pd.read_csv("https://raw.githubusercontent.com/andriygav/PSAD/master/labs/lab2/data/2.1.csv", delimiter='\,')


,with vaccine,without vaccine
0,h,h
1,h,s
2,h,s
3,h,h
4,h,h
5,h,s
6,h,h
7,h,s
8,h,s
9,h,s


In [29]:
# data_pre['without vaccine']
#data = pd.Dataframe({'without vaccine': data_pre['without vaccine'], 'with vaccine': data_pre['with vaccine']})

0     h
1     s
2     s
3     h
4     h
5     s
6     h
7     s
8     s
9     s
10    s
11    h
12    s
13    h
14    h
15    s
16    h
17    h
18    s
19    h
20    s
21    h
22    h
23    h
24    s
25    s
26    s
27    s
28    h
29    s
Name: without vaccine, dtype: object

По данным сразу видно, что заболевших с вакциной - всего 5, а без вакцины - 16, т.е. есть все основания полагать, что вакцина действенна

Поменяем выборки местами, т.е. сначала без вакцины с $p_1 = 0.5$, а потом с вакциной с $p_2 = 0.2$, чтобы было легче формулировать задачу как на лекции)
1. Формальная постановка задачи: 
\begin{align*}
\text{$x_{1i}, x_{2i}$ поступают парами}\\
\text{рассматриваем различающиеся } &\text{пары: (1,0) и (0,1)}\\
\text{относительный риск:  } & u = \dfrac{p_1(1-p_2)}{p_2(1-p_1)}\\
& u <(=,>) 1 \Leftrightarrow p_1 <(=,>) p_2\\
\text{коридор отклонений:  }& u_L \leq 1 < u_U\\
& H_0: u > u_U \\
& H_1: u \leq u_L \\
\text{статистика: } & d_m(X_1^m, X_2^m) = \sum\limits_{i=1}^m(1-X_{i1})X_{i2}
\end{align*}

2. Оценка среднего количества дней для принятия решения: у нас имеется формула для мат.ожидания числа дней на принятие решения
$$\mathbb{E}_u(n) = \dfrac{L(u)\ln B + (1-L(u))\ln A}{\frac{u}{1+u}\ln\frac{u_U(1+u_L)}{u_L(1+u_U)} +  \frac{1}{1+u}\ln\frac{1+u_L}{1+u_U}}\cdot\left[p_1(1-p_2)+p_2(1-p_1)\right]^{-1}$$
где 
$$ L(u) = \dfrac{A^h-1}{A^h-B^h} - \text{оперативная характеристика}$$
$h$ определяется из решения уравнения
$$ \dfrac{u}{1+u} = \dfrac{1 - \left(\frac{1+u_L}{1+u_U}\right)^h}{\left(\frac{u_U(1+u_L)}{u_L(1+u_U)}\right)^h - \left(\frac{1+u_L}{1+u_U}\right)^h}$$
$$ A = \dfrac{1-\beta}{\alpha}, \hspace{0.5cm} B = \dfrac{\beta}{1-\alpha}$$

In [14]:
def f(x):
    s = (1 - ((1 + uL)/(1 +uU))**x)/(((uU*(1+uL))/(uL*(1+uU)))**x - ((1 + uL)/(1 +uU))**x ) - u/(1+u)
    return s

In [21]:
alpha, beta = 0.05, 0.2
A = (1-beta)/alpha
B = beta/(1-alpha)
print('alpha =', alpha, 'beta =', beta, '\nA =', A, 'B =', B)
p1, p2 = 0.5, 0.2
u = (p1*(1-p2))/(p2*(1-p1))
print('u =', u)
uL, uU = 3, 5

h = scipy.optimize.root_scalar(f, bracket=[-10, 10], method='brentq').root
print('h =', h)
L = (A**h -1)/(A**h - B**h)
print('L =', L)
En = (L * np.log(B) + (1-L) * np.log(A))/ (u/(1+u)*np.log((uU*(1+uL))/(uL*(1+uU))) + 1/(1+u) * np.log((1 + uL)/(1 +uU))) / (p1*(1-p2) + p2*(1-p1))
print('E(n) =', En)

alpha = 0.05 beta = 0.2 
A = 16.0 B = 0.2105263157894737
u = 4.0
h = -0.1507455403932004
L = 0.5633666050906951
E(n) = 208.29888592820492


Как-то очень много шагов ожидается

3. Выяснение рабоспособности вакцины.

Преобразуем данные в пары (1,0)

In [24]:
values = (data_pre.values == 's').astype(int)
values

array([[0, 0],
       [0, 1],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [0, 0],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0],
       [1, 0],
       [0, 1],
       [0, 0],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 0],
       [1, 1]])

In [25]:
deltas = (values[:, 0] < values[:, 1])
deltas

array([False,  True,  True, False, False,  True, False,  True,  True,
        True, False, False,  True, False, False,  True, False, False,
        True, False,  True, False, False, False, False,  True, False,
        True, False, False])